In [1]:
from haversine import haversine

from DB import *
db_connection = connect_db()

In [2]:
main = load_data_from_rds('생태통로', db_connection)
main

,번호,위도,경도
0,1495,35.36019,128.06375
1,1277,37.03408,128.40661
2,1278,36.89832,128.30563
3,1279,36.89687,128.30581
4,1090,36.71051,127.42222
...,...,...,...
492,1089,36.71278,127.42056
493,1087,36.71722,127.41778
494,1088,36.71222,127.42528
495,1091,36.76750,127.47333


In [12]:
roadkill = pd.read_csv('../data/variables/한국도로공사_로드킬데이터_2021.csv', encoding='cp949')
roadkill

,본부명,지사명,노선명,구간,방 향,5km,발생건수,위도,경도
0,수도권,경기광주,제2중부선,330~335,통영,330,5,37.283059,127.385590
1,수도권,경기광주,제2중부선,335~340,하남,335,3,37.315325,127.347275
2,수도권,경기광주,중부선,330~335,통영,330,3,37.283193,127.384920
3,수도권,경기광주,중부선,340~345,통영,340,3,37.353642,127.320539
4,수도권,경기광주,중부선,325~330,하남,325,3,37.248957,127.421370
...,...,...,...,...,...,...,...,...,...
103,대구경북,영주,중앙선,180~185,부산,180,5,36.478684,128.617816
104,대구경북,영주,중앙선,225~230,춘천,225,3,36.809892,128.543780
105,대구경북,청송,당진영덕선,115~120,영덕,115,3,36.433562,128.608436
106,부산경남,고성,중부선,15~20,통영,15,3,34.975219,128.381369


In [4]:
# A-B 지점 간 거리(km) 계산 함수
def distance(A_lat, A_long, B_lat, B_long): # A 지점 위도 및 경도, B 지점 위도 및 경도
    A = (A_lat, A_long)
    B = (B_lat, B_long)
    dist = haversine(A, B, unit='km')
    return dist

In [5]:
# 생태통로 반경 2.3km 이내 발생한 로드킬
dist_list = []
i_list = []
j_list = []
for i, m_row in main.iterrows(): # 각 생태통로 반복
    for j, rk_row in roadkill.iterrows(): # 각 로드킬 반복
        dist = distance(m_row['위도'], m_row['경도'], rk_row['위도'], rk_row['경도'])
        if dist <= 2.3:
            dist_list.append(dist)
            i_list.append(i)
            j_list.append(j)

In [6]:
dist_df = pd.DataFrame({'main_idx': i_list,
                        'roadkill_idx': j_list,
                        'distance': dist_list})
dist_df

,main_idx,roadkill_idx,distance
0,4,41,1.378719
1,6,8,2.146326
2,29,86,1.217189
3,33,88,0.397620
4,68,46,1.878145
5,69,48,1.866237
6,100,11,1.553199
7,101,12,1.797967
8,111,7,2.292573
9,113,86,1.688272


In [7]:
main['주변 로드킬 빈도'] = 0 # 0으로 초기화

for i, d_row in dist_df.iterrows():
    main.loc[d_row['main_idx'], '주변 로드킬 빈도'] += roadkill.loc[d_row['roadkill_idx'], ' 발생건수 '] # 한 생태통로 주변에 로드킬 발생 건이 여러 건일 경우를 고려해, 누적 빈도 값을 저장

print(len(main[main['주변 로드킬 빈도'] != 0]))

28


## 최종 데이터

In [9]:
df = main[['번호', '주변 로드킬 빈도']]
df

,번호,주변 로드킬 빈도
0,1495,0
1,1277,0
2,1278,0
3,1279,0
4,1090,5
...,...,...
492,1089,5
493,1087,5
494,1088,5
495,1091,0


## RDS에 데이터 업로드

In [8]:
table = load_data_from_rds('변수관계설정', db_connection)
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도
0,1001,1.903039,0,6,X,NaN,NaN
1,1002,0.120319,1,5,X,4.0,4.0
2,1004,0.047211,1,2,산지,1.0,1.0
3,1005,0.233820,1,2,X,4.0,4.0
4,1006,2.172428,0,4,하천,3.0,3.0
...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0
493,1562,2.835246,1,5,산지,6.0,27.0
494,1564,0.886170,1,4,X,1.0,1.0
495,1567,9.125829,1,5,하천,4.0,5.0


In [10]:
table = table.merge(df, on='번호')
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도
0,1001,1.903039,0,6,X,NaN,NaN,0
1,1002,0.120319,1,5,X,4.0,4.0,0
2,1004,0.047211,1,2,산지,1.0,1.0,0
3,1005,0.233820,1,2,X,4.0,4.0,0
4,1006,2.172428,0,4,하천,3.0,3.0,0
...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4
493,1562,2.835246,1,5,산지,6.0,27.0,0
494,1564,0.886170,1,4,X,1.0,1.0,0
495,1567,9.125829,1,5,하천,4.0,5.0,0


In [11]:
upload_data_to_rds(table, '변수관계설정', db_connection)